## Method used in a KDD 2009 competition

We will cover the feature selection approach undertaken by data scientists at the University of Melbourne in the [KDD 2009](http://www.kdd.org/kdd-cup/view/kdd-cup-2009) data science competition. The task consisted in predicting churn based on a dataset with a huge number of features.

The authors describe this procedure as an aggressive non-parametric feature selection procedure that is based in contemplating the relationship between the feature and the target. Therefore, this method should be classified as a filter method.

**The procedure consists in the following steps**:

For each categorical variable:

    1) Separate into train and test

    2) Determine the mean value of the target within each label of the categorical variable using the train set

    3) Use that mean target value per label as the prediction (using the test set) and calculate the roc-auc.

For each numerical variable:

    1) Separate into train and test
    
    2) Divide the variable into 100 quantiles

    3) Calculate the mean target within each quantile using the training set

    4) Use that mean target value / bin as the prediction (using the test set) and calculate the roc-auc


The authors quote the following advantages of the method:

- Speed: computing mean and quantiles is direct and efficient
- Stability respect to scale: extreme values for continuous variables do not skew the predictions
- Comparable between categorical and numerical variables
- Accommodation of non-linearities

See my notes at the end of the notebook for a discussion on the method.

**Important**
The authors here use the roc-auc, but in principle, we could use any metric, including those valid for regression.

**Reference**:
[Predicting customer behaviour: The University of Melbourne's KDD Cup Report. Miller et al. JMLR Workshop and Conference Proceedings 7:45-55](http://www.mtome.com/Publications/CiML/CiML-v3-book.pdf)

## How to do it?

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split

from sklearn.metrics import roc_auc_score

In [ ]:
# load the titanic dataset
usecols = ['pclass', 'survived', 'sex', 'age', 'sibsp', 'parch', 'fare', 'cabin', 'embarked']
data = pd.read_csv('/content/drive/MyDrive/Colab_Notebooks/Bai tap lap/Mon_hoc_2/titanic.csv', usecols=usecols)
data.shape

(1309, 9)

In [ ]:
data.head()

,pclass,survived,sex,age,sibsp,parch,fare,cabin,embarked
0,1,1,female,29.0000,0,0,211.3375,B5,S
1,1,1,male,0.9167,1,2,151.5500,C22,S
2,1,0,female,2.0000,1,2,151.5500,C22,S
3,1,0,male,30.0000,1,2,151.5500,C22,S
4,1,0,female,25.0000,1,2,151.5500,C22,S


In [ ]:
# Variable preprocessing:

# then I will narrow down the different cabins by selecting only the
# first letter, which represents the deck in which the cabin was located

# captures first letter of string (the letter of the cabin)
data['cabin'] = data['cabin'].str[0]
data['cabin'].unique()

array(['B', 'C', 'E', 'D', 'A', nan, 'T', 'F', 'G'], dtype=object)

### Feature selection on categorical variables

First, I will demonstrate the feature selection procedure over categorical variables. The Titanic dataset contains 4 categorical variables, which are Sex, Pclass, Cabin and Embarked.

**Important**

In all feature selection procedures, it is good practice to select the features by examining only the training set. And this is to avoid overfit.

In [ ]:
# separate train and test sets

# I will only use the categorical variables and the target

X_train, X_test, y_train, y_test = train_test_split(
    data[['pclass', 'sex', 'embarked', 'cabin', 'survived']],
    data['survived'],
    test_size=0.3,
    random_state=0)

X_train.shape, X_test.shape

((916, 5), (393, 5))

In [ ]:
y_train.info()

<class 'pandas.core.series.Series'>
Int64Index: 916 entries, 501 to 684
Series name: survived
Non-Null Count  Dtype
--------------  -----
916 non-null    int64
dtypes: int64(1)
memory usage: 14.3 KB


### Replace categories by target mean

In [ ]:
# function that determines the target mean per category

def mean_encoding(df_train, df_test, categorical_vars):

    # temporary copy of the original dataframes
    df_train_temp = df_train.copy()
    df_test_temp = df_test.copy()

    # iterate over each variable
    for col in categorical_vars:

        # make a dictionary of categories, target-mean pairs
        target_mean_dict = df_train.groupby([col], dropna=False)['survived'].mean().to_dict()
        print(target_mean_dict)
        # replace the categories by the mean of the target
        df_train_temp[col] = df_train[col].map(target_mean_dict)
        df_test_temp[col] = df_test[col].map(target_mean_dict)

    # drop the target from the daatset
    df_train_temp.drop(['survived'], axis=1, inplace=True)
    df_test_temp.drop(['survived'], axis=1, inplace=True)

    # return  remapped datasets
    return df_train_temp, df_test_temp

In [ ]:
categorical_vars = ['pclass', 'sex', 'embarked', 'cabin']
X_train_enc, X_test_enc = mean_encoding(X_train, X_test, categorical_vars)
X_train_enc.head()

{1: 0.6173913043478261, 2: 0.43617021276595747, 3: 0.25903614457831325}
{'female': 0.7283582089552239, 'male': 0.18760757314974183}
{'C': 0.553072625698324, 'Q': 0.37349397590361444, 'S': 0.3389570552147239, nan: 1.0}
{'A': 0.5294117647058824, 'B': 0.7619047619047619, 'C': 0.5633802816901409, 'D': 0.71875, 'E': 0.71875, 'F': 0.6666666666666666, 'G': 0.5, 'T': 0.0, nan: 0.30484330484330485}


,pclass,sex,embarked,cabin
501,0.436170,0.728358,0.338957,0.304843
588,0.436170,0.728358,0.338957,0.304843
402,0.436170,0.728358,0.553073,0.304843
1193,0.259036,0.187608,0.373494,0.304843
686,0.259036,0.728358,0.373494,0.304843


In [ ]:
X_test_enc.head()

,pclass,sex,embarked,cabin
1139,0.259036,0.187608,0.338957,0.304843
533,0.436170,0.728358,0.338957,0.304843
459,0.436170,0.187608,0.338957,0.304843
1150,0.259036,0.187608,0.338957,0.304843
393,0.436170,0.187608,0.338957,0.304843


The strings were replaced by the target mean.

### Determine the roc-auc using the variable values as input

In [ ]:
# now, we calculate a roc-auc value, using the encoded variables
# as predictions

roc_values = []

for feature in categorical_vars:

    roc_values.append(roc_auc_score(y_test, X_test_enc[feature]))

In [ ]:
# I make a series for easy visualisation

m1 = pd.Series(roc_values)
m1.index = categorical_vars
m1.sort_values(ascending=False)

sex         0.749959
pclass      0.670096
cabin       0.628809
embarked    0.593358
dtype: float64

We can see, that all the features are important, because the roc_auc for all of them is higher than 0.5.

Sex seems to be the most important feature to predict survival, as its roc_auc is the highest.

As you see, this is a very powerful, yet straightforward approach to feature selection.

## Feature Selection on numerical variables

The procedure is exactly the same, but it requires one additional first step which is to divide the continuous variable into bins.

The authors of the method divide the variable in 100 quantiles, that is 100 bins. In principle, you could divide the variable in less bins. Here I will divide the variable in 5 bins only.

I will work with the numerical variables Age and Fare.

In [ ]:
# separate train and test sets
X_train, X_test, y_train, y_test = train_test_split(
    data[['age', 'fare', 'survived']],
    data['survived'],
    test_size=0.3,
    random_state=0)

X_train.shape, X_test.shape

((916, 3), (393, 3))

In [ ]:
# fill missing values

X_train = X_train.fillna(0)
X_test = X_test.fillna(0)

#### Bin variable Age

In [ ]:
# Let's divide Age in 10 bins. We use the qcut (quantile cut)
# function from pandas indicating that 11 cutting points (q),
# thus 10 bins.

# retbins= True indicates that I want to capture the limits of
# each interval (so I can then use them to cut the test set)

X_train['age_binned'], intervals = pd.qcut(
    X_train['age'],
    q = 5,
    labels=False,
    retbins=True,
    precision=3,
    duplicates='drop',
)

X_train[['age_binned', 'age']].head(10)

,age_binned,age
501,1,13.0
588,1,4.0
402,3,30.0
1193,0,0.0
686,1,22.0
971,0,0.0
117,3,30.0
540,1,2.0
294,4,49.0
261,3,35.0


In [ ]:
# count the number of distinct bins

X_train['age_binned'].nunique()

5

In [ ]:
# display the bins

X_train['age_binned'].unique()

array([1, 3, 0, 4, 2])

In [ ]:
# now I use the interval limits calculated in the previous cell to
# bin the testing set

X_test['age_binned'] = pd.cut(x = X_test['age'], bins=intervals, labels=False)

X_test[['age_binned', 'age']].head(10)

,age_binned,age
1139,3.0,38.0
533,1.0,21.0
459,4.0,42.0
1150,NaN,0.0
393,2.0,25.0
1189,1.0,4.0
5,4.0,48.0
231,4.0,52.0
330,4.0,57.0
887,NaN,0.0


#### Bin Variable Fare

In [ ]:
# train
X_train['fare_binned'], intervals = pd.qcut(
    X_train['fare'],
    q=5,
    labels=False,
    retbins=True,
    precision=3,
    duplicates='drop',
)

# test
X_test['fare_binned'] = pd.cut(x = X_test['fare'], bins=intervals, labels=False)

In [ ]:
X_test['fare_binned'].nunique()

5

In [ ]:
X_test.isnull().sum()

age             0
fare            0
survived        0
age_binned     90
fare_binned     4
dtype: int64

In [ ]:
# test shows some missing data. The missing values in
# the test set appear when the original values are outside
# the boundaries of the invervals determined in the train set
# that is, values that are smaller or bigger than the min and max
# values from the train set

# to speed out the demo, I will just replace them by 0 in this notebook

X_train = X_train.fillna(0)
X_test = X_test.fillna(0)

### Replace bins with target mean

In [ ]:
# now we use our previous function to encode the variables
# with the target mean

binned_vars = ['age_binned', 'fare_binned']

X_train_enc, X_test_enc = mean_encoding(
    X_train[binned_vars+['survived']], X_test[binned_vars+['survived']], binned_vars)

X_train_enc.head()

{0: 0.2903225806451613, 1: 0.3942307692307692, 2: 0.3885350318471338, 3: 0.44041450777202074, 4: 0.4127906976744186}
{0: 0.29347826086956524, 1: 0.21195652173913043, 2: 0.37158469945355194, 3: 0.4340659340659341, 4: 0.6174863387978142}


,age_binned,fare_binned
501,0.394231,0.371585
588,0.394231,0.434066
402,0.440415,0.371585
1193,0.290323,0.293478
686,0.394231,0.293478


### Determine roc-auc using encodings

In [ ]:
# now, we calculate a roc-auc value, using the encoded variables
# as predictions

roc_values = []

for feature in binned_vars:

    roc_values.append(roc_auc_score(y_test, X_test_enc[feature]))

In [ ]:
# I make a series for easy visualisation

m1 = pd.Series(roc_values)
m1.index = binned_vars
m1.sort_values(ascending=False)

fare_binned    0.704496
age_binned     0.500442
dtype: float64

Fare, is a much better predictor of Survival. Age produces a random output, the roc-auc is 0.5.


**Some thoughts**

The authors mention that by using this method, you are able to compare directly numerical with categorical variables. In a sense this is true, however we need to keep in mind, that categorical variables may or may not (and typically they will not) show the same percentage of observations per label. However, when we divide a numerical variable into quantile bins, we guarantee that each bin shows the same percentage of observations.

Alternatively, instead of binning into quantiles, we can bin into equal-distance bins.

That is all for this lecture, I hope you enjoyed it and see you in the next one!